In [1]:
#最ajob表出来，加入前7/14天 end_time start_time 等。

import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'

d_list= ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
          '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
          '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
          '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
          '180711', '180712', '180713', '180714', '180715']
#47

day_all = ['171211', '171212', '171213', '171214', '171215', '171216', '171217', '171218', '171219', '171220', \
           '171221', '171222', '171223', '171224', '171225', '171226', '171227', '171228', '171229', '171230', \
           '171231', '180101', '180102', '180104', '180105', '180106', '180107', '180108', '180109', '180110', \
           '180111', '180112', '180113', '180114', '180115', '180116', '180117', '180118', '180119', '180120', \
           '180121', '180122', '180123', '180124', '180125', '180126', '180127', '180128', '180129', '180130', '180131',\
           '180201', '180202', '180203', '180204', '180205', '180206', '180207', '180208', '180209', '180210',\
           '180211', '180212', '180213', '180214', '180215', '180216', '180217', '180218', '180219', '180220', \
           '180221', '180222', '180223', '180224', '180225', '180226', '180227', '180228', \
           '180301', '180302', '180303', '180304', '180305', '180306', '180307', '180308', '180309', '180310', \
           '180311', '180312', '180313', '180314', '180315', '180316', '180317', '180318', '180319', '180320',\
           '180321', '180322', '180323', '180324', '180325', '180326', '180327', '180328', '180329', '180330', '180331', \
           '180401', '180402', '180403', '180404', '180405', '180406', '180407', '180408', '180409', '180410', \
           '180411', '180412', '180413', '180414', '180415', '180416', '180417','180418', '180419', '180420',\
           '180421', '180422', '180423', '180424', '180425', '180426', '180427', '180428', '180429', '180430',\
           '180501', '180502', '180503', '180504', '180505', '180506', '180507', '180508', '180509', '180510',\
           '180511', '180512', '180513', '180514', '180515', '180516', '180517', '180518', '180519', '180520',\
           '180521', '180522', '180523', '180524', '180525', '180526', '180527', '180528', '180529', '180530',\
           '180531','180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715']

In [2]:
def get_interval_mint(s):
    if s is None:
        return 0
    value = int(s[0:5])
    mytype = s[5:]
    if mytype=='M':
        value = value
    elif mytype=='H':
        value = value*60
    return value

def f_replace_oa(s):
    return s.replace('oa','0a')

def not_find_end_time(a):
    if a==-1:
        return 1
    else:
        return 0

def get_avg_run_start_mint(s):
    if s is None:
        return None
    else:
        h = s//10000
        m = (s%10000)//100
        s = s%100
        return h*60+m+s/60
f_sum_std = ('min','max','median','mean','std')

In [28]:
#for day in d_list:
day = '180706'
train_src = pd.read_csv('./ajob/ajob_%s.txt'%day,low_memory=False)
day_i  = day_all.index(day)

day_4 = int('20'+day+'040000')
train_src = train_src[train_src.order_time<day_4]
train_src = train_src[train_src.avg_start_time.notnull()]
train_src = train_src[train_src.avg_start_time<400000]
train_src = train_src[train_src.order==day]



train_src.drop('confirm_flag',axis=1,inplace=True)
train_src.drop('job_id',axis=1,inplace=True)
train_src.drop('from_time',axis=1,inplace=True)
train_src.drop('next_time',axis=1,inplace=True)
train_src.drop('to_time',axis=1,inplace=True)
train_src.drop('delete_flag',axis=1,inplace=True)
train_src.drop('rerun_counter',axis=1,inplace=True)
train_src.drop('prev_odate_rerun_counter',axis=1,inplace=True)
train_src.drop('cpu_time',axis=1,inplace=True)
train_src.drop('cpu_id',axis=1,inplace=True)

#train_src['nodegroup']= train_src['nodegroup'].fillna('none')
#train_src['priority'] = train_src['priority'].apply(f_replace_oa)
#train_src['interval'] = train_src['interval'].apply(get_interval_mint)
#train_src['avg_runtime'] = train_src['avg_runtime'].apply(get_avg_run_stat_mint)
train_src['avg_start_time'] = train_src['avg_start_time'].apply(get_avg_run_start_mint)


def get_job_n_day_before_start_time(n,job_list):

        day = day_all[day_i - n]
        print(day)
        day_begain = int('20'+day+'000000')
        day_8 = int('20'+day+'040000') #实例化要在8点前，排除手工追数情况;
        if n ==0:
            df =train_src
        else:
            df = pd.read_csv('./ajob/ajob_%s.txt'%day,low_memory=False)
        df = df[df.end_time.notnull()]
        #print(df.shape)
        df = df[(df.order_time>day_begain)&(df.order_time<day_8)]
        #print(df.shape)


        dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')

        def f_get_mint_bef(x):
            dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
            dt = (dat_t-dat_start).total_seconds()//60
            return dt

        #df['end_time'] = df['end_time'].apply(int).apply(f_get_mint_bef)
        df['start_time'] = df['start_time'].apply(int).apply(f_get_mint_bef)

        df_start = df[['job_name','start_time']]
        #df_end = df[['job_name','end_time']]

        df_start_dict = df_start.groupby('job_name').start_time.min().T.to_dict()
        #df_end_dict = df_end.groupby('job_name').end_time.min().T.to_dict()

        start_not_findjob = list(set(job_list)-set(df_start_dict.keys()))
        #end_not_findjob = list(set(job_list)-set(df_end_dict.keys()))

        for job in start_not_findjob:
            df_start_dict[job] = -1
#         for job in end_not_findjob:
#             df_end_dict[job] = -1

        return job_list.apply(lambda x:df_start_dict[x])

    #统计前 n  天日期 完成时间
for k in range(1,22,1):
    #print(k)
    train_src['day_%d_bef_start_time'%k] = get_job_n_day_before_start_time(k,train_src['job_name'])
    

day_1_k_bef_start_time = ['day_%d_bef_start_time'%k for k in range(1,40)]

180707
180706
180705
180704
180703
180702
180701
180630
180629
180628
180627
180626
180625
180624
180623
180622
180621
180620
180619
180618
180617


In [33]:
day_int = int(day)
train_src = train_src[train_src.odate==day_int]
for i in range(7,22):
    day_1_k_sum = day_1_k_bef_start_time[:i]
    train_src['day_1_k_avg_start_time'] = train_src[day_1_k_sum].apply('mean',axis=1)
    train_src['day_k_start_time_min'] = train_src[day_1_k_sum].apply(min, axis=1)
    train_src['day_k_no_start_time'] = train_src['day_k_start_time_min'].apply(not_find_end_time)
    
    train_src['avg_1_k_sub_avg'] = train_src[['day_1_k_avg_start_time','avg_start_time']].apply(lambda x:abs(x[1]-x[0]),axis=1)
    
    train_src = train_src.round(2)
    
    print(i,train_src['avg_1_k_sub_avg'].mean(),train_src[train_src['day_k_no_start_time']==0]['avg_1_k_sub_avg'].mean())


7 43.7189086389 33.9196065701
8 40.2581004384 30.8136799599
9 39.3635363322 30.3198409456
10 37.1951406509 28.2925506874
11 35.1951590842 26.165392551
12 32.8071764487 23.5687342428
13 29.7813919095 20.1767509526
14 27.5903518209 17.6855692563
15 27.0374005143 17.0162157502
16 27.5116339824 17.4152300142
17 25.9521201032 15.7358529789
18 25.0440302368 14.5582739862
19 24.2004891801 13.5144579052
20 23.8573758849 12.9250017215
21 23.7159611661 12.4528427594


In [ ]:
7 43.7188989575 33.9195944998
8 40.2581004384 30.8136799599
9 39.3635363322 30.3198409456
10 37.1951406509 28.2925506874
11 35.1951590842 26.165392551
12 32.8071764487 23.5687342428
13 29.7813919095 20.1767509526
14 27.5903518209 17.6855692563
15 27.0374005143 17.0162157502
16 27.5116339824 17.4152300142
17 25.9521201032 15.7358529789
18 25.0440302368 14.5582739862
19 24.2004891801 13.5144579052
20 23.8573758849 12.9250017215
21 23.7159611661 12.4528427594

In [39]:
#for day in d_list:
day = '180710'
train_src = pd.read_csv('./ajob/ajob_%s.txt'%day,low_memory=False)
day_i  = day_all.index(day)

day_4 = int('20'+day+'040000')
train_src = train_src[train_src.order_time<day_4]
train_src = train_src[train_src.avg_start_time.notnull()]
train_src = train_src[train_src.avg_start_time<400000]
train_src = train_src[train_src.odate==int(day)]



train_src.drop('confirm_flag',axis=1,inplace=True)
train_src.drop('job_id',axis=1,inplace=True)
train_src.drop('from_time',axis=1,inplace=True)
train_src.drop('next_time',axis=1,inplace=True)
train_src.drop('to_time',axis=1,inplace=True)
train_src.drop('delete_flag',axis=1,inplace=True)
train_src.drop('rerun_counter',axis=1,inplace=True)
train_src.drop('prev_odate_rerun_counter',axis=1,inplace=True)
train_src.drop('cpu_time',axis=1,inplace=True)
train_src.drop('cpu_id',axis=1,inplace=True)

#train_src['nodegroup']= train_src['nodegroup'].fillna('none')
#train_src['priority'] = train_src['priority'].apply(f_replace_oa)
#train_src['interval'] = train_src['interval'].apply(get_interval_mint)
#train_src['avg_runtime'] = train_src['avg_runtime'].apply(get_avg_run_stat_mint)
train_src['avg_start_time'] = train_src['avg_start_time'].apply(get_avg_run_start_mint)


def get_job_n_day_before_start_time(n,job_list):

        day = day_all[day_i - n]
        print(day)
        day_begain = int('20'+day+'000000')
        day_8 = int('20'+day+'040000') #实例化要在8点前，排除手工追数情况;
        if n ==0:
            df =train_src
        else:
            df = pd.read_csv('./ajob/ajob_%s.txt'%day,low_memory=False)
        df = df[df.end_time.notnull()]
        #print(df.shape)
        df = df[(df.order_time>day_begain)&(df.order_time<day_8)]
        #print(df.shape)


        dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')

        def f_get_mint_bef(x):
            dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
            dt = (dat_t-dat_start).total_seconds()//60
            return dt

        #df['end_time'] = df['end_time'].apply(int).apply(f_get_mint_bef)
        df['start_time'] = df['start_time'].apply(int).apply(f_get_mint_bef)

        df_start = df[['job_name','start_time']]
        #df_end = df[['job_name','end_time']]

        df_start_dict = df_start.groupby('job_name').start_time.min().T.to_dict()
        #df_end_dict = df_end.groupby('job_name').end_time.min().T.to_dict()

        start_not_findjob = list(set(job_list)-set(df_start_dict.keys()))
        #end_not_findjob = list(set(job_list)-set(df_end_dict.keys()))

        for job in start_not_findjob:
            df_start_dict[job] = -1
#         for job in end_not_findjob:
#             df_end_dict[job] = -1

        return job_list.apply(lambda x:df_start_dict[x])

    #统计前 n  天日期 完成时间
for k in range(2,22):
    #print(k)
    train_src['day_%d_bef_start_time'%k] = get_job_n_day_before_start_time(k,train_src['job_name'])
    

day_1_k_bef_start_time = ['day_%d_bef_start_time'%k for k in range(2,22)]

train_src['avg_start_time'] = train_src['avg_start_time'].apply(lambda x:x*21)
train_src['day_2_21_bef_start_time_sum'] = train_src[day_1_k_bef_start_time].apply(sum,axis=1)

train_src['day_1_bef_start_int'] = train_src[['avg_start_time','day_2_21_bef_start_time_sum']].apply(lambda x:abs(x[0]-x[1]),axis=1)




day_bef = day_all[day_i - 1]
day_bef_start = datetime.datetime.strptime(str('20'+day_bef+'000000'),'%Y%m%d%H%M%S')

def f_cnv_int_to_time(int_t):
    
    return((day_bef_start+datetime.timedelta(minutes=int_t)).strftime('%Y%m%d%H%M%S'))
    
train_src['day_1_bef_start_time'] = train_src['day_1_bef_start_int'].apply(f_cnv_int_to_time)

train_src_save = train_src[['job_name','day_1_bef_start_time','day_1_bef_start_int']]




180708
180707
180706
180705
180704
180703
180702
180701
180630
180629
180628
180627
180626
180625
180624
180623
180622
180621
180620
180619


In [40]:
day_bef

'180709'

In [48]:
day_bef_df = train_src = pd.read_csv('./ajob/ajob_%s.txt'%day_bef,low_memory=False)

day_bef_df = day_bef_df[['order_id','job_name','v_date','avg_runtime','start_time','end_time']]

day_bef_df =day_bef_df.merge(train_src_save,on=['job_name'],how='left')

day_bef_df = day_bef_df[day_bef_df.start_time.notnull()]

day_bef_df['start_time'] =day_bef_df['start_time'].apply(int)

day_bef_df = day_bef_df.round(2)
#day_bef_df.to_csv('submit_%s.csv'%day_bef,index=False)

In [49]:
def get_time_del(x):
    d1 = datetime.datetime.strptime(str(x[0]),'%Y%m%d%H%M%S')
    d2 = datetime.datetime.strptime(str(x[1]),'%Y%m%d%H%M%S')
    dt = (d2-d1).total_seconds()//60
    return dt
day_bef_sum = day_bef_df[day_bef_df.start_time.notnull()]
day_bef_sum = day_bef_sum[day_bef_sum.day_1_bef_start_time.notnull()]

day_bef_sum = day_bef_sum[['start_time','day_1_bef_start_time']]



AttributeError: 'Series' object has no attribute 'meam'

In [50]:
import numpy as np

np.mean(day_bef_sum.apply(get_time_del,axis=1))

157.23002752883025

In [4]:
import pandas as pd
import datetime
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'

d_list= [  '180625', '180626', '180627', '180628', '180629', '180630',\
          '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708']

In [7]:
df_list = []
for day in d_list:
    print(day,end=' ')
    train_gen = pd.read_csv('./ajob/ajob_1_%s.csv'%day)
    
    #x = train_gen[cols_train]
    #y = train_gen['day_0_bef_end_time']
    
    train_gen = train_gen[(train_gen.day_0_bef_end_time>240)&(train_gen.day_0_bef_end_time<1530)&(train_gen.day_1_bef_start_time.notnull())]
    train_gen['sub_valid'] = train_gen[['day_0_bef_end_time','day_1_bef_start_time','avg_runtime']].apply(lambda x:abs(x[0]-x[1]-x[2]),axis=1)
    print(' ',train_gen['sub_valid'].mean())

180625   62.7709215982
180626   82.1206685251
180627   70.6067584775
180628   85.5125416822
180629   123.84218447
180630   117.236535013
180701   99.6965106969
180702   150.353861691
180703   102.522794463
180704   70.6125006451
180705   93.2382138143
180706   101.294197111
180707   126.65772879
180708   67.5168117995
